In [1]:
#Importar librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report,roc_auc_score, recall_score,accuracy_score, mean_squared_error, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
#Se cargan los datos ya procesados
df=pd.read_excel('prueba.xlsx')

In [14]:
#Se mapea la variable tiro para la correcta lectura de los algoritmos
mapeo = {'triple': 4, 'doble': 3, 'zona': 2, 'TL': 1, 'mate': 0}

# Aplicar el mapeo a la columna 'tiro'
df['tiro'] = df['tiro'].map(mapeo)

In [19]:
#Variables mas importantes
variables = ['x', 'y', 'cuarto',
       'tiempo', 'reloj_posesion', 'local', 'diferencia', 'distancia',
       'angulo', 'tiro', 'rating_TL', 'rating_doble', 'rating_mate',
       'rating_triple', 'rating_zona', 'cluster', 'canasta_anterior']

In [11]:
#Se prueban 29 algoritmos con Lazypredict
y = df['canasta']
X = df[variables]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LazyClassifier()
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)

100%|█████████████████████████████████████████████████████████████████████████████████| 29/29 [54:45<00:00, 113.30s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LGBMClassifier                     0.67               0.67     0.67      0.67   
AdaBoostClassifier                 0.66               0.66     0.66      0.66   
XGBClassifier                      0.66               0.66     0.66      0.66   
SVC                                0.66               0.66     0.66      0.66   
GaussianNB                         0.66               0.66     0.66      0.66   
NearestCentroid                    0.65               0.65     0.65      0.65   
LogisticRegression                 0.65               0.65     0.65      0.65   
LinearDiscriminantAnalysis         0.65               0.65     0.65      0.65   
RidgeClassifierCV                  0.65               0.65     0.65      0.65   
CalibratedClassifierCV             0.65               0.65     0.65      0.65   
LinearSVC                   

In [ ]:
# Realizar el Gridsearchcv sobre los 5 algoritmos elegidos
# Definir el conjunto de parámetros para la búsqueda en la cuadrícula para cada algoritmo
param_grid = {
    'XGBoost': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5,7]},
    'LightBoost': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5,7]},
    'GradientBoosting': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5,7]},
    'RandomForest': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5,7]},
    'MLPClassifier': {'hidden_layer_sizes': [(50,100,150), (50,100,150,200)], 'activation': ['relu', 'tanh'], 'learning_rate': ['constant', 'adaptive'], 'max_iter': [100, 200]}
}

# Variable 'canasta' es tu variable objetivo
X = df[variables]
y = df['canasta']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
valores = X_test['tipo']
X_train = X_train.drop(columns=['tipo'],axis=1)
X_test = X_test.drop(columns=['tipo'],axis=1)
for nombre_algoritmo, clf in algoritmos.items():
    
    # Configurar GridSearchCV para el algoritmo actual
    grid_search = GridSearchCV(clf, param_grid[nombre_algoritmo], cv=5, scoring='roc_auc')

    # Ajustar el modelo utilizando GridSearchCV
    grid_search.fit(X_train, y_train)

    # Obtener el mejor modelo después de la búsqueda en la cuadrícula
    clf_best = grid_search.best_estimator_
    probabilidades = clf_best.predict_proba(X_test)
    pts_pred = (probabilidades[:, 1] * valores)
    pts_real = (y_test * valores)
    y_pred = clf_best.predict(X_test)

    # Imprimir resultados
    print(f"Resultados para {nombre_algoritmo}:")
    print('Puntos reales: ', pts_real.sum(), 'Puntos predichos: ', pts_pred.sum())
    print('RMSE: ', np.sqrt(((pts_pred.sum() - pts_real.sum()) ** 2) / len(y_test)))
   
    # Calcular y mostrar el classification report, accuracy, precision y f1-score globales
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'F1-Score: {f1:.4f}')
    print(f'Recall: {recall:.4f}')
    print('ROC-AUC: ', roc_auc_score(y_test, y_pred))
    
    # Si el algoritmo tiene atributo feature_importances_, mostrar las características importantes
    if hasattr(clf_best, 'feature_importances_'):
        feature_importances = clf_best.feature_importances_
        feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
        print("\nFeature Importances:")
        print(feature_importance_df)

    print("=" * 60 + "\n")

In [22]:
#Se evaluan los 5 algoritmos elegidos tras el gridsearchcv 
# y se obtienen los resultados de todas las metricas utilizadas con objetivo de ROC-AUC
# Definir algoritmos
algoritmos = {
    'XGBoost': XGBClassifier(n_estimators=50, max_depth=3),
    'LightBoost': LGBMClassifier(n_estimators=200, max_depth=7),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=50, max_depth=5),
    'RandomForest': RandomForestClassifier(n_estimators=200, max_depth=7),
    'MLPClassifier': MLPClassifier(hidden_layer_sizes=(200, 150, 100, 50),activation='tanh',learning_rate='adaptive',max_iter=500)
}

# Variable 'canasta' es tu variable objetivo
X = df[variables]
y = df['canasta']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
valores = X_test['tipo']
X_train = X_train.drop(columns=['tipo'],axis=1)
X_test = X_test.drop(columns=['tipo'],axis=1)

# Iterar sobre algoritmos
for nombre_algoritmo, clf in algoritmos.items():
    # Ajustar el modelo
    clf.fit(X_train, y_train)

    # Evaluar el modelo
    probabilidades = clf.predict_proba(X_test)
    pts_pred = (probabilidades[:, 1] * valores)
    pts_real = (y_test * valores)
    y_pred = clf.predict(X_test)

    # Imprimir resultados
    print(f"Resultados para {nombre_algoritmo}:")
    print('Puntos reales: ', pts_real.sum(), 'Puntos predichos: ', pts_pred.sum())
    print('RMSE: ', np.sqrt(((pts_pred.sum() - pts_real.sum()) ** 2) / len(y_test)))
    print('RMSE modificado: ',  np.sqrt(mean_squared_error(y_test, probabilidades[:,1])))
    print('RMSE modificado2: ', np.sqrt(mean_squared_error(y_test, y_pred*1)))
    print('RMSE modificado3: ', np.sqrt(mean_squared_error(y_test*valores, y_pred*1*valores)))
    print('RMSE modificado4: ', np.sqrt(mean_squared_error(y_test*valores, probabilidades[:,1]*valores)))
    print('Promedio de la diferencia probabilidades: ', (((((y_test*1)-probabilidades[:, 1])*valores).abs()).sum())/len(y_test))
    print('Promedio de la diferencia predicciones: ', (((((y_test*1)-y_pred)*valores).abs()).sum())/len(y_test))
    
    # Calcular y mostrar el classification report, accuracy, precision y f1-score globales
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'F1-Score: {f1:.4f}')
    print(f'Recall: {recall:.4f}')
    print('ROC-AUC: ', roc_auc_score(y_test, y_pred))
    
    # Si el algoritmo tiene atributo feature_importances_, mostrar las características importantes
    if hasattr(clf, 'feature_importances_'):
        feature_importances = clf.feature_importances_
        feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
        print("\nFeature Importances:")
        print(feature_importance_df)

    print("=" * 60 + "\n")

Resultados para XGBoost:
Puntos reales:  21548 Puntos predichos:  21718.027503628284
RMSE:  1.1508854592578164
RMSE modificado:  0.45952645
RMSE modificado2:  0.5677340938175843
RMSE modificado3:  1.292206442188331
RMSE modificado4:  1.037261299887859
Promedio de la diferencia probabilidades:  0.9059299289844647
Promedio de la diferencia predicciones:  0.6993952167140108
Accuracy: 0.6777
Precision: 0.7206
F1-Score: 0.6759
Recall: 0.6363
ROC-AUC:  0.6801405602192763

Feature Importances:
            Feature  Importance
5              tiro        0.83
4            angulo        0.07
3         distancia        0.03
1    reloj_posesion        0.02
0                 x        0.01
6  canasta_anterior        0.01
2        diferencia        0.01

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Resultados para LightBoost:
Puntos reales:  21548 Puntos predichos:  21708.633048003074
RMSE:  1.0872960860918481
RMSE mo

In [4]:
#Se cargaron los datos asociados a los partidos de la temporada 23/24 del Obradoiro
obradoiro = pd.read_excel('partidos_obra_23_24.xlsx')

In [7]:
# Determinar los identificadores de partido de la temporada 23/24 del Obradoiro para dejar esos datos en el conjunto test
lim_superior = obradoiro['partido'].max()
lim_inferior = obradoiro['partido'].min()

In [23]:
#Se dividieron en conjunto de train y test, dejando en test los datos asociados a los 23 partidos del Obraodiro
entrenar = df[(df['partido']<lim_inferior)|(df['partido']>lim_superior)]
testear = df[(df['partido']>=lim_inferior)&(df['partido']<=lim_superior)]

In [24]:
#Se separaron en x e y, además se obtuvieron los valores de tipo de tiro para luego eliminarlo de los X_train y test
X_train = entrenar[variables]
y_train = entrenar['canasta']
X_test = testear[variables]
y_test = testear['canasta']
valores = X_test['tipo']
X_train = X_train.drop(columns=['tipo'],axis=1)
X_test = X_test.drop(columns=['tipo'],axis=1)

In [ ]:
# Entrenar los datos con LGBClassifier con los párametros óptimos
clf = LGBClassifier(n_estimators=200, max_depth=7)
clf.fit(X_train, y_train)

In [29]:
#Se renombraron variables de equipo local y visitante
obradoiro=obradoiro.rename(columns={'visitante': 'away'})
obradoiro=obradoiro.rename(columns={'local': 'home'})

In [30]:
#Se creó una nueva variable para luego poder saber las acciones de cada equipo
obradoiro['team'] = obradoiro.apply(lambda row: row['home'] if row['equipo'] == row['equipo_local'] else row['away'], axis=1)

In [34]:
#Se eliminaron todas las columnas menos 3 del dataframe anterior
obra=obradoiro.drop(columns=['Unnamed: 0', 'id', 'jugador', 'canasta', 'x', 'y',
       'cuarto', 'tiempo', 'tipo', 'zona', 'accion', 'puntuacion_local',
       'puntuacion_visitante', 'asistencia', 'reloj_posesion', 'equipo_local',
       'equipo_visitante', 'home', 'away'])

In [39]:
#Se eliminaron variables sin importancia
datos_monbus = pd.merge(testear,obra,on=['partido','equipo']).drop_duplicates()
datos_monbus = datos_monbus.drop(columns=['Unnamed: 0'],axis=1)

In [44]:

# Calcularon resultados para el equipo de las 23 jornadas jugadas en liga Endesa del Obradoiro

# Agrupar por 'partido' e 'equipo'
grupos = datos_monbus.groupby(['partido', 'equipo'])


# Iterar sobre los grupos
for (partido, equipo), grupo in grupos:
    # Obtener los valores de 'tipo' y 'y' para el grupo actual
    valores_canasta = grupo['tipo']
    y = grupo['canasta']
    X = grupo[variables]
    # Calcular los puntos reales para el grupo actual
    pts_reales = (valores_canasta * y).sum()

    # Suponiendo que X es el conjunto de características para el grupo actual
    # Calcular las probabilidades para el grupo actual
    probabilidades = clf.predict_proba(X.loc[grupo.index])

    # Calcular los puntos predichos para el grupo actual
    pts_pred = (probabilidades[:, 1] * valores_canasta).sum()


# Imprimir los resultados totales
    print(f'Equipo, partido: {equipo,partido}')
    print(f'Equipo que anotó: {grupo["team"].iloc[0]}') 
    print(f'Puntos reales: {pts_reales}')
    print(f'Puntos predichos: {pts_pred}')

Equipo, partido: (4226, 13103568)
Equipo que anotó: Monbus Obradoiro
Puntos reales: 77
Puntos predichos: 75.56175774335861
Equipo, partido: (4236, 13103568)
Equipo que anotó: Bàsquet Girona
Puntos reales: 78
Puntos predichos: 78.90287985280156
Equipo, partido: (4226, 13103608)
Equipo que anotó: Monbus Obradoiro
Puntos reales: 86
Puntos predichos: 79.31607005000114
Equipo, partido: (4231, 13103608)
Equipo que anotó: Lenovo Tenerife
Puntos reales: 97
Puntos predichos: 81.28968468308449
Equipo, partido: (4226, 13103613)
Equipo que anotó: Monbus Obradoiro
Puntos reales: 74
Puntos predichos: 76.17250061035156
Equipo, partido: (4227, 13103613)
Equipo que anotó: Coviran Granada
Puntos reales: 77
Puntos predichos: 78.01398780941963
Equipo, partido: (4224, 13103617)
Equipo que anotó: Unicaja
Puntos reales: 90
Puntos predichos: 80.77170272171497
Equipo, partido: (4226, 13103617)
Equipo que anotó: Monbus Obradoiro
Puntos reales: 78
Puntos predichos: 81.80197298526764
Equipo, partido: (4226, 13103